In [3]:
import numpy as np
import pandas as pd
import seaborn as sns

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [59]:
iris = sns.load_dataset("iris")

In [60]:
# 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width),
# 꽃받침길이(petal_length), 꽃받침폭(petal_width)의 평균, 표준편차 등
# 기초통계량(describe())을 구하시오.
iris.groupby("species").agg(["mean","std"]).describe()

sepal_length           sepal_width           petal_length            \
              mean       std        mean       std         mean       std   
count     3.000000  3.000000    3.000000  3.000000       3.0000  3.000000   
mean      5.843333  0.501513    3.057333  0.338453       3.7580  0.398490   
std       0.795061  0.142262    0.336822  0.035438       2.0907  0.198973   
min       5.006000  0.352490    2.770000  0.313798       1.4620  0.173664   
25%       5.471000  0.434330    2.872000  0.318147       2.8610  0.321787   
50%       5.936000  0.516171    2.974000  0.322497       4.2600  0.469911   
75%       6.262000  0.576025    3.201000  0.350781       4.9060  0.510903   
max       6.588000  0.635880    3.428000  0.379064       5.5520  0.551895   

      petal_width            
             mean       std  
count    3.000000  3.000000  
mean     1.199333  0.192596  
std      0.896735  0.084750  
min      0.246000  0.105386  
25%      0.786000  0.151569  
50%      1.326000  0.197753  
75%      1.676000  0.236201  
max      2.026000  0.274650

In [61]:
# b. 3분위수(Q3)와 1분위수(Q1)의 차이보다 1.5배가 크거나 작은 데이터는 이상치이다.즉,
# Q1 - 1.5 * (Q3 - Q1) 보다 작은 데이터
# Q3 + 1.5 * (Q3 - Q1) 보다 큰 데이터
# 이 이상치를 제거하고 위의 4가지 항목에 대해서 평균, 표준편차를 구하시오.

s = iris[iris.species=="setosa"]["sepal_length"]
s.mean(), s.std()

(5.005999999999999, 0.3524896872134512)

In [55]:
q1, q3 = s.quantile(.25), s.quantile(.75)
iqr = q3 - q1
q1, q3, iqr

(4.8, 5.2, 0.40000000000000036)

In [56]:
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr
s[(s < lower) | (s > upper)] = np.nan
s.isnull().sum()

0

In [57]:
s.mean(), s.std()

(5.005999999999999, 0.3524896872134512)

In [97]:
# 위의 내용을 함수로
def get_new_stat(s):
    q1, q3 = s.quantile(.25), s.quantile(.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    s[(s < lower) | (s > upper)] = np.nan
    outlier = s.isnull().sum() != 0
    #return outlier, s.mean(), s.std()
    return outlier, np.round(s.mean(),2), np.round(s.std(),4)

In [98]:
species_list, feature_list, mean_list, std_list = [],[],[],[]
outlier_list, new_mean_list, new_std_list = [],[],[]
for species in iris.species.unique():
    for feature in iris.columns[:-1]:
        s = iris[iris.species == species][feature]
        species_list.append(species)
        feature_list.append(feature)
        mean_list.append(np.round(s.mean(), 2))
        std_list.append(np.round(s.std(), 4))

        outlier, new_mean, new_std = get_new_stat(s)
        outlier_list.append(outlier)
        new_mean_list.append(new_mean)
        new_std_list.append(new_std)

In [99]:
df = pd.DataFrame({
    'species': species_list, 'feature': feature_list,
    'mean': mean_list, 'std': std_list,
    'outlier': outlier_list, 'new_mean': new_mean_list, 'new_std': new_std_list
})
df.set_index(['species','feature'], inplace=True)
df

mean     std  outlier  new_mean  new_std
species    feature                                               
setosa     sepal_length  5.01  0.3525    False      5.01   0.3525
           sepal_width   3.43  0.3791     True      3.43   0.3203
           petal_length  1.46  0.1737     True      1.46   0.1291
           petal_width   0.25  0.1054     True      0.23   0.0859
versicolor sepal_length  5.94  0.5162    False      5.94   0.5162
           sepal_width   2.77  0.3138    False      2.77   0.3138
           petal_length  4.26  0.4699     True      4.29   0.4378
           petal_width   1.33  0.1978    False      1.33   0.1978
virginica  sepal_length  6.59  0.6359     True      6.62   0.5935
           sepal_width   2.97  0.3225     True      2.96   0.2603
           petal_length  5.55  0.5519    False      5.55   0.5519
           petal_width   2.03  0.2747    False      2.03   0.2747

In [63]:
#1. 타이타닉 오픈
titanic = sns.load_dataset("titanic")
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [65]:
# 2. 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 열인 category1 열을 만드시오.
#       category1 카테고리는 다음과 같이 정의됨
#       2.1) 20살이 넘으면 성별을 그대로 사용한다.
#       2.2) 20살 미만이면 성별에 관계없이 “child”라고 한다.

# 2. 열추가
# 2.1 추가열에 값 추가

titanic["category1"] = titanic.apply(lambda r: r.sex if r.age >= 20 else "child", axis =1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


In [71]:
# 타이타닉호의 승객 중 나이를 명시하지 않은 고객은 나이를 명시한 고객의 평균 나이 값
# 이 되도록 titanic 데이터프레임을 고치시오.

# 평균 나이값 찾기
titanic.age.fillna(titanic.age.mean(), inplace=True)
titanic.head()


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,male
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,female
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,female
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,female
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,male


In [83]:
# 성별, 선실(class)별, 출발지(embark_town)별 생존율을 구하시오.
# titanic.groupby(["sex", "class","embark_town"])[["survived"]].mean()
titanic.groupby("sex")[["survived"]].mean()
#titanic.groupby("sex")["survived"].mean()

,survived
sex,
female,0.742038
male,0.188908


In [74]:
titanic.pivot_table("survived","class")

,survived
class,
First,0.629630
Second,0.472826
Third,0.242363


In [73]:
titanic.pivot_table("survived","embark_town")

,survived
embark_town,
Cherbourg,0.553571
Queenstown,0.389610
Southampton,0.336957


In [76]:
titanic.pivot_table("survived",["sex","class"])

survived
sex    class           
female First   0.968085
       Second  0.921053
       Third   0.500000
male   First   0.368852
       Second  0.157407
       Third   0.135447

In [75]:
titanic.pivot_table("survived",["sex","class"],"embark_town")

embark_town    Cherbourg  Queenstown  Southampton
sex    class                                     
female First    0.976744    1.000000     0.958333
       Second   1.000000    1.000000     0.910448
       Third    0.652174    0.727273     0.375000
male   First    0.404762    0.000000     0.354430
       Second   0.200000    0.000000     0.154639
       Third    0.232558    0.076923     0.128302

In [79]:
# d. 타이타닉호 승객을 ‘미성년자’, ‘청년’, ‘중년’, ‘장년’, ‘노년’ 나이 그룹으로 나누고,
# 각 그룹별 생존율을 구하시오.
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]

titanic["age_cat"] = pd.cut(titanic["age"], bins, labels=labels)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,age_group,age_cat
886,0,2,male,27.000000,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male,청년,청년
887,1,1,female,19.000000,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child,미성년자,미성년자
888,0,3,female,29.699118,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child,청년,청년
889,1,1,male,26.000000,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male,청년,청년
890,0,3,male,32.000000,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male,중년,중년


In [80]:
#titanic.groupby(["age_group"])[["survived"]].mean()
titanic.pivot_table("survived","age_cat")

,survived
age_cat,
미성년자,0.424242
청년,0.334152
중년,0.423237
장년,0.355932
노년,0.200000


In [81]:
titanic.pivot_table("survived",["sex","age_cat"])

survived
sex    age_cat          
female 미성년자     0.671233
       청년       0.723881
       중년       0.779070
       장년       0.941176
male   미성년자     0.228261
       청년       0.142857
       중년       0.225806
       장년       0.119048
       노년       0.200000

In [84]:
# e. qcut 명령으로 세 개의 나이 그룹을 만들고, 나이 그룹별 남녀 성비와 생존율을 구하시오.
titanic["age_group"] = pd.qcut(titanic.age, 3, labels=["A1","A2","A3"])
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,age_group,age_cat
886,0,2,male,27.000000,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male,A2,청년
887,1,1,female,19.000000,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child,A1,미성년자
888,0,3,female,29.699118,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child,A2,청년
889,1,1,male,26.000000,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male,A2,청년
890,0,3,male,32.000000,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male,A3,중년


In [86]:
titanic.groupby("age_group")[["survived"]].mean()

,survived
age_group,
A1,0.411960
A2,0.335526
A3,0.405594


In [88]:
# gender : 남성이면 1, 여성이면 0
titanic["gender"] = titanic.apply(lambda r: 1 if r.sex == "mail" else 0, axis=1) 
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,age_group,age_cat,gender
886,0,2,male,27.000000,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male,A2,청년,0
887,1,1,female,19.000000,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child,A1,미성년자,0
888,0,3,female,29.699118,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child,A2,청년,0
889,1,1,male,26.000000,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male,A2,청년,0
890,0,3,male,32.000000,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male,A3,중년,0


In [89]:
titanic.pivot_table("gender","age_group")

,gender
age_group,
A1,0
A2,0
A3,0


In [95]:
titanic.pivot_table(["gender","survived"], "age_group")

,gender,survived
age_group,,
A1,0,0.411960
A2,0,0.335526
A3,0,0.405594


In [5]:
# 3. Mile Per Gallon - sns.load_dataset('mpg')
mpg = sns.load_dataset("mpg")
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [6]:
# a. 배기량(displacement) 대비 마력(horsepower) 열(hp_per_cc)을 추가하시오.

#내가한거
#mpg["displacement"] = 0
#mpg["horsepower"] = 0
#mpg["hp_per_cc"] = 0
#mpg.head()

mpg["hp_per_cc"] = mpg.horsepower / mpg.displacement
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,0.423453
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,0.471429
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,0.471698
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,0.493421
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,0.463576


In [7]:
# b. name으로부터 manufacturer(제조사)와 모델을 추출하여
# 새로운 열 manufacturer와 model을 추가하고, name 열은 삭제하시오.
mpg["manufacturer"] = mpg.name.apply(lambda x: x.split()[0])
mpg["model"]= mpg.name.apply(lambda x: ' '.join(x.split()[1:]))
#mpg.drop(["name"],axis=1,inplace=True)
del mpg["name"]
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacturer,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,0.423453,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,0.471429,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,0.471698,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,0.493421,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,0.463576,ford,torino


In [8]:
# c. 엔진의 실린더(cylinders) 갯수별 연비(mpg)의 평균을 구하시오.
mpg.groupby(["cylinders"])[["mpg"]].mean()


,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


In [9]:
# 3가지 형태의 동일한 결과
# mpg[["mpg","cylinders"]].groupby("cylinders").mean()
mpg.pivot_table("mpg","cylinders")

,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


In [10]:
# 생산지(origin)별 배기량 대비 마력(hp_per_cc)의 평균을 구하시오.
# mpg.groupby(["origin"])[["hp_per_cc"]].mean()
mpg.pivot_table("hp_per_cc","origin")

,hp_per_cc
origin,
europe,0.740005
japan,0.789485
usa,0.512063


In [11]:
# 모델이 5개 이상인 제조사에 대하여
# 연비(mpg)의 평균이 가장 좋은 제조사 Top 5를 구하시오.
# mpg.groupby(["manufacturer"]).count()

mpg.manufacturer.value_counts()

ford             51
chevrolet        43
plymouth         31
amc              28
dodge            28
toyota           25
datsun           23
buick            17
pontiac          16
volkswagen       15
honda            13
mercury          11
mazda            10
oldsmobile       10
fiat              8
peugeot           8
audi              7
chrysler          6
vw                6
volvo             6
renault           5
saab              4
subaru            4
opel              4
chevy             3
bmw               2
cadillac          2
maxda             2
mercedes-benz     2
triumph           1
vokswagen         1
mercedes          1
hi                1
capri             1
chevroelt         1
toyouta           1
nissan            1
Name: manufacturer, dtype: int64

In [15]:
s = mpg.manufacturer.value_counts()
#s[s>5].index
#s[s>5].index.tolist()
big = s[s>5].index.tolist()
print(big)

['ford', 'chevrolet', 'plymouth', 'amc', 'dodge', 'toyota', 'datsun', 'buick', 'pontiac', 'volkswagen', 'honda', 'mercury', 'mazda', 'oldsmobile', 'fiat', 'peugeot', 'audi', 'chrysler', 'vw', 'volvo']


In [21]:
mpg[mpg.mpg > 40].head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacturer,model
244,43.1,4,90.0,48.0,1985,21.5,78,europe,0.533333,volkswagen,rabbit custom diesel
309,41.5,4,98.0,76.0,2144,14.7,80,europe,0.775510,vw,rabbit
322,46.6,4,86.0,65.0,2110,17.9,80,japan,0.755814,mazda,glc
324,40.8,4,85.0,65.0,2110,19.2,80,japan,0.764706,datsun,210
325,44.3,4,90.0,48.0,2085,21.7,80,europe,0.533333,vw,rabbit c (diesel)


In [16]:
# in 연산자 대신에 isin() method 사용
mpg[mpg.manufacturer.isin(big)].head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacturer,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,0.423453,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,0.471429,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,0.471698,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,0.493421,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,0.463576,ford,torino


In [18]:
bigmpg = mpg[mpg.manufacturer.isin(big)]
bigmpg.pivot_table("mpg", "manufacturer").sort_values(by="mpg", ascending=False).head()

,mpg
manufacturer,
vw,39.016667
honda,33.761538
datsun,31.113043
mazda,30.860000
volkswagen,29.106667


In [19]:
mpg.groupby("manufacturer").filter(lambda x: x.manufacturer.count() >= 5).head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacturer,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,0.423453,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,0.471429,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,0.471698,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,0.493421,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,0.463576,ford,torino


In [20]:
new_mpg = mpg.groupby("manufacturer").filter(lambda x: x.manufacturer.count() >= 5)
new_mpg.pivot_table("mpg","manufacturer").sort_values(by="mpg", ascending=False).head()

,mpg
manufacturer,
vw,39.016667
honda,33.761538
renault,32.880000
datsun,31.113043
mazda,30.860000
